In [25]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()



In [26]:
from IPython import get_ipython
from IPython.display import display

In [50]:
# Clone dev branch for the latest model
!git clone -b dev --recursive https://github.com/shaoanlu/fewshot-face-translation-GAN.git

fatal: destination path 'fewshot-face-translation-GAN' already exists and is not an empty directory.


In [51]:
# There are import errors under keras == 2.2.5
!pip install --upgrade tensorflow keras --quiet


In [53]:
%cd fewshot-face-translation-GAN

/content/fewshot-face-translation-GAN


In [54]:
# Download pre-trined weights
!gdown https://drive.google.com/uc?id=1ThyUhXkVY5WJIRbik_Jpx9vOqEQ4jEZw
!gdown https://drive.google.com/uc?id=1cVkPjTxqQXo0pyLQHw_Yhngc4GO6KB5Q

Downloading...
From: https://drive.google.com/uc?id=1ThyUhXkVY5WJIRbik_Jpx9vOqEQ4jEZw
To: /content/fewshot-face-translation-GAN/encoder.h5
100% 6.26M/6.26M [00:00<00:00, 87.3MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1cVkPjTxqQXo0pyLQHw_Yhngc4GO6KB5Q
From (redirected): https://drive.google.com/uc?id=1cVkPjTxqQXo0pyLQHw_Yhngc4GO6KB5Q&confirm=t&uuid=fc24bf8a-5830-495c-b800-23e0dd01c3f7
To: /content/fewshot-face-translation-GAN/decoder.h5
100% 126M/126M [00:01<00:00, 65.3MB/s]


In [31]:
!mkdir weights
!mv decoder.h5 weights/decoder.h5
!mv encoder.h5 weights/encoder.h5

mkdir: cannot create directory ‘weights’: File exists


In [32]:
!ls

colab_demo.ipynb    fewshot-face-translation-GAN  preprocess.py  utils
configs		    images			  __pycache__	 weights
data		    models.py			  README.md
face_toolbox_keras  networks			  train.py


## Configuration

In [55]:
from utils.config_loader import load_yaml
config = load_yaml("configs/config_inference.yaml"); config

{'dir_weights': './weights',
 'identity_extractor': 'inceptionresnetv1',
 'input_size': 224,
 'latent_dim': 512,
 'separate_adain': True,
 'additional_emb': True,
 'use_nwg': False,
 'nc_in': 3}

## Load GAN model

In [35]:
import warnings
warnings.filterwarnings("ignore")

In [36]:
# Navigate to the models.py file in the cloned repository and modify it.
# Replace the line importing Beta from tensorflow.contrib
!sed -i "s/from tensorflow.contrib.distributions import Beta/from tensorflow.compat.v1.distributions import Beta/" /content/fewshot-face-translation-GAN/fewshot-face-translation-GAN/models.py

In [56]:
file_path = "/content/fewshot-face-translation-GAN/models.py"

with open(file_path, "r") as file:
    code = file.read()

# Remove old contrib import
code = code.replace("from tensorflow.contrib.distributions import Beta",
                    "import tensorflow_probability as tfp\nBeta = tfp.distributions.Beta")

with open(file_path, "w") as file:
    file.write(code)

print("Updated import for Beta distribution using TensorFlow Probability.")


Updated import for Beta distribution using TensorFlow Probability.


In [57]:
file_path = "/content/fewshot-face-translation-GAN/networks/nn_blocks.py"

with open(file_path, "r") as f:
    code = f.read()

# Fix invalid import
code = code.replace(
    "from keras.layers.advanced_activations import LeakyReLU",
    "from keras.layers import LeakyReLU"
)

with open(file_path, "w") as f:
    f.write(code)

print("Fixed LeakyReLU import in nn_blocks.py")


Fixed LeakyReLU import in nn_blocks.py


In [39]:
file_path = "/content/fewshot-face-translation-GAN/networks/instance_normalization.py"

with open(file_path, "r") as f:
    code = f.read()

# Fix Layer + InputSpec import
code = code.replace(
    "from keras.engine import Layer, InputSpec",
    "from tensorflow.keras.layers import Layer, InputSpec"
)

# Ensure all other keras.* are tensorflow.keras.*
code = code.replace("from keras.", "from tensorflow.keras.")
code = code.replace("import keras.", "import tensorflow.keras.")

with open(file_path, "w") as f:
    f.write(code)

print("Fixed imports in instance_normalization.py")


Fixed imports in instance_normalization.py


In [40]:
!find /content/fewshot-face-translation-GAN -type f -name "*.py" -exec sed -i 's/tf\.get_default_graph()/tf.compat.v1.get_default_graph()/g' {} \;

print("Replaced tf.get_default_graph() with tf.compat.v1.get_default_graph()")


Replaced tf.get_default_graph() with tf.compat.v1.get_default_graph()


In [41]:
file_path = "/content/fewshot-face-translation-GAN/networks/instance_normalization.py"

with open(file_path, "r") as f:
    code = f.read()

# Fix Layer + InputSpec import
code = code.replace(
    "from keras.engine import Layer, InputSpec",
    "from tensorflow.keras.layers import Layer, InputSpec"
)

# Fix the import path for get_custom_objects
code = code.replace(
    "from tensorflow.keras.utils.generic_utils import get_custom_objects",
    "from tensorflow.keras.utils import get_custom_objects"
)


# Ensure all other keras.* are tensorflow.keras.*
code = code.replace("from keras.", "from tensorflow.keras.")
code = code.replace("import keras.", "import tensorflow.keras.")

with open(file_path, "w") as f:
    f.write(code)

print("Fixed imports in instance_normalization.py")

Fixed imports in instance_normalization.py


In [42]:
file_path = "/content/fewshot-face-translation-GAN/networks/discriminator.py"

with open(file_path, "r") as f:
    code = f.read()

# Fix invalid import for LeakyReLU
code = code.replace(
    "from keras.layers.advanced_activations import LeakyReLU",
    "from keras.layers import LeakyReLU"
)

with open(file_path, "w") as f:
    f.write(code)

print("Fixed LeakyReLU import in discriminator.py")

Fixed LeakyReLU import in discriminator.py


In [58]:
file_path = "/content/fewshot-face-translation-GAN/networks/instance_normalization.py"

with open(file_path, "r") as f:
    code = f.read()

# Ensure tensorflow is imported as tf
if "import tensorflow as tf" not in code:
    code = "import tensorflow as tf\n" + code

# Fix Layer + InputSpec import
code = code.replace(
    "from keras.engine import Layer, InputSpec",
    "from tensorflow.keras.layers import Layer, InputSpec"
)

# Fix the import path for get_custom_objects if it exists and is incorrect
code = code.replace(
    "from tensorflow.keras.utils.generic_utils import get_custom_objects",
    "from tensorflow.keras.utils import get_custom_objects"
)

# Ensure all other keras.* are tensorflow.keras.* imports
# This is a broad replacement, be careful if it causes other issues,
# but it's necessary for the migration.
code = code.replace("from keras.", "from tensorflow.keras.")
code = code.replace("import keras.", "import tensorflow.keras.")

# Find the call method and replace the int_shape line
# This is a more targeted fix for the specific error location
# Look for lines that look like 'input_shape = ...int_shape(inputs)'
import re
# Use regex to find lines getting input_shape using int_shape
# This pattern is more robust to whitespace variations
code = re.sub(r'input_shape\s*=\s*.*?int_shape\(inputs\)',
              'input_shape = tf.shape(inputs)',
              code)

# Remove any leftover keras.backend imports if they exist and aren't used elsewhere
code = re.sub(r'import keras\.backend\s*as\s*K', '', code)
code = re.sub(r'from keras\s*import\s*backend\s*as\s*K', '', code)


with open(file_path, "w") as f:
    f.write(code)

print("Fixed int_shape usage in InstanceNormalization.call method using tf.shape.")

Fixed int_shape usage in InstanceNormalization.call method using tf.shape.


In [47]:
file_path = "/content/fewshot-face-translation-GAN/networks/instance_normalization.py"

with open(file_path, "r") as f:
    code = f.read()

# Ensure tensorflow is imported as tf
if "import tensorflow as tf" not in code:
    code = "import tensorflow as tf\n" + code

# Fix Layer + InputSpec import
code = code.replace(
    "from keras.engine import Layer, InputSpec",
    "from tensorflow.keras.layers import Layer, InputSpec"
)

# Fix the import path for get_custom_objects if it exists and is incorrect
code = code.replace(
    "from tensorflow.keras.utils.generic_utils import get_custom_objects",
    "from tensorflow.keras.utils import get_custom_objects"
)

# Ensure all other keras.* are tensorflow.keras.* imports
# This is a broad replacement, be careful if it causes other issues,
# but it's necessary for the migration.
code = code.replace("from keras.", "from tensorflow.keras.")
code = code.replace("import keras.", "import tensorflow.keras.")

# Explicitly replace keras.backend.int_shape with tf.shape within the code,
# especially targeting the call method if possible, but a general replacement
# should also work if keras.backend is only used for int_shape.
# Let's specifically target the int_shape usage.
code = code.replace("keras.backend.int_shape(inputs)", "tf.shape(inputs)")
# Also handle if it's just K.int_shape(inputs) after an import 'import keras.backend as K'
code = code.replace("K.int_shape(inputs)", "tf.shape(inputs)")


# Remove any leftover keras.backend imports if they exist and aren't used elsewhere
code = re.sub(r'import keras\.backend\s*as\s*K', '', code)
code = re.sub(r'from keras\s*import\s*backend\s*as\s*K', '', code)


with open(file_path, "w") as f:
    f.write(code)

print("Fixed int_shape usage in InstanceNormalization.call method using tf.shape and removed backend imports.")

Fixed int_shape usage in InstanceNormalization.call method using tf.shape and removed backend imports.


In [77]:
import tensorflow as tf
import re

file_path_norm = "/content/fewshot-face-translation-GAN/networks/instance_normalization.py"
file_path_models = "/content/fewshot-face-translation-GAN/models.py"

# --- Fix Instance Normalization ---
try:
    with open(file_path_norm, "r") as f:
        code = f.read()

    # Ensure tensorflow is imported as tf
    if "import tensorflow as tf" not in code:
        code = "import tensorflow as tf\n" + code

    # Fix Layer + InputSpec import
    code = code.replace(
        "from keras.engine import Layer, InputSpec",
        "from tensorflow.keras.layers import Layer, InputSpec"
    )

    # Fix the import path for get_custom_objects if it exists and is incorrect
    code = code.replace(
        "from tensorflow.keras.utils.generic_utils import get_custom_objects",
        "from tensorflow.keras.utils import get_custom_objects"
    )

    # Ensure all other keras.* are tensorflow.keras.* imports
    code = code.replace("from keras.", "from tensorflow.keras.")
    code = code.replace("import keras.", "import tensorflow.keras.")

    # Replace keras.backend.int_shape or K.int_shape with tf.shape
    code = code.replace("keras.backend.int_shape(inputs)", "tf.shape(inputs)")
    code = code.replace("K.int_shape(inputs)", "tf.shape(inputs)")

    # --- FIX: Replace the core calculation block in the call method ---
    # We need to find the 'call' method definition.
    call_method_pattern = r"(def\s+call\(self,\s+inputs,\s*training\s*=\s*None\):.*?)(\n\s*def\s+|$)" # Capture the call method block

    call_method_match = re.search(call_method_pattern, code, re.DOTALL) # Use DOTALL to match across lines

    if call_method_match:
        call_method_block = call_method_match.group(1)

        # Define the correct calculation block with proper indentation
        # We'll try to find the indentation from the 'def call' line itself
        indent_match = re.match(r"(\s*)def\s+call", call_method_block)
        indent = indent_match.group(1) if indent_match else "" # Get leading whitespace

        # We'll replace the *entire* content of the call method with the correct version
        # to be sure no old code remains. This is safer than trying to surgically
        # replace parts of the block.
        new_call_method_content = f"""
{indent}    # Get input shape and rank symbolically
{indent}    input_shape = tf.shape(inputs)
{indent}    input_rank = tf.rank(inputs)
{indent}
{indent}    # Instance normalization axes exclude batch (0) and channels (last)
{indent}    # reduction_axes should cover spatial dimensions (1 to rank-2)
{indent}    # Ensure rank is at least 3 (batch, spatial, channel) for instance norm
{indent}    tf.debugging.assert_greater_equal(input_rank, 3, message="Input rank must be at least 3 for instance normalization.")
{indent}    # Calculate reduction axes dynamically based on rank
{indent}    reduction_axes = tf.range(1, input_rank - 1) # Exclude batch (0) and channel (rank-1)
{indent}
{indent}    # Calculate mean and variance over the reduction axes
{indent}    # Use tf.nn.moments for numerical stability and symbolic compatibility
{indent}    mean, variance = tf.nn.moments(inputs, axes=reduction_axes, keepdims=True)
{indent}
{indent}    # Add epsilon for numerical stability
{indent}    variance = variance + self.epsilon
{indent}
{indent}    # Normalization formula: (x - mean) / sqrt(variance + epsilon) * gamma + beta
{indent}    inv = tf.math.rsqrt(variance)
{indent}    normalized = (inputs - mean) * inv
{indent}
{indent}    # Apply scale (gamma) and offset (beta) if they exist
{indent}    if self.scale:
{indent}        normalized = normalized * self.gamma
{indent}    if self.center:
{indent}        normalized = normalized + self.beta
{indent}
{indent}    return normalized
"""
        # Replace the old call method block with the new one.
        # Find the exact span of the original call method block (including the def line)
        call_method_span = call_method_match.span()
        code = code[:call_method_span[0]] + new_call_method_content + code[call_method_span[1]:]

        print("Replaced the entire InstanceNormalization.call method with a standard implementation.")

        # Final check: search for 'len(' within the modified code for the call method
        call_method_match_after = re.search(call_method_pattern, code, re.DOTALL)
        if call_method_match_after:
             call_method_block_after = call_method_match_after.group(1)
             if 'len(' in call_method_block_after:
                 print("Warning: Found 'len(' still present in the modified InstanceNormalization.call method.")
             else:
                 print("Checked: 'len(' not found in the modified InstanceNormalization.call method.")


    else:
         print("Could not find the 'call' method definition in InstanceNormalization.py")


    # Remove any leftover keras.backend imports if they exist and aren't used elsewhere
    code = re.sub(r'import keras\.backend\s*as\s*K', '', code)
    code = re.sub(r'from keras\s*import\s+backend\s*as\s*K', '', code)

    with open(file_path_norm, "w") as f:
        f.write(code)

    print("Attempted comprehensive fix for InstanceNormalization.call method logic.")

except FileNotFoundError:
    print(f"Error: File not found at {file_path_norm}")
except Exception as e:
    print(f"An unexpected error occurred while fixing InstanceNormalization: {e}")


# --- Remove generic exception in models.py ---
try:
    with open(file_path_models, "r") as f:
        code_models = f.read()

    # Find the __init__ method block
    init_method_pattern = r"(def\s+__init__\(self,\s+config\):.*?)(\n\s*def\s+|$)"
    init_method_match = re.search(init_method_pattern, code_models, re.DOTALL)

    if init_method_match:
        init_method_block = init_method_match.group(1)

        # Replace the try...except block with just the content inside the try
        # assuming the content inside is the correct initialization logic
        # This pattern looks for the try...except around build_encoder and build_decoder
        try_except_pattern = r"(\s*)try:(\s*self\.encoder\s*=\s*self\.build_encoder\(\)\s*self\.decoder\s*=\s*self\.build_decoder\(\))(\s*)except:(\s*raise Exception\(\"Error building networks\.\"\))"
        updated_init_block = re.sub(try_except_pattern, r"\1\2", init_method_block, flags=re.DOTALL)

        if updated_init_block != init_method_block:
             # Replace the old init block with the updated one
            init_method_span = init_method_match.span()
            code_models = code_models[:init_method_span[0]] + updated_init_block + code_models[init_method_span[1]:]
            print("Removed generic try...except block in models.py __init__.")
        else:
             print("Could not find the generic try...except block in models.py __init__ to remove.")

    else:
        print("Could not find the __init__ method definition in models.py")


    with open(file_path_models, "w") as f:
        f.write(code_models)

except FileNotFoundError:
    print(f"Error: File not found at {file_path_models}")
except Exception as e:
    print(f"An unexpected error occurred while fixing models.py: {e}")

Replaced the entire InstanceNormalization.call method with a standard implementation.
Attempted comprehensive fix for InstanceNormalization.call method logic.
Could not find the generic try...except block in models.py __init__ to remove.


In [78]:
from models import FaceTranslationGANInferenceModel

In [79]:
model = FaceTranslationGANInferenceModel(config=config)

Exception: Error building networks.

In [ ]:
from face_toolbox_keras.models.verifier.face_verifier import FaceVerifier
fv = FaceVerifier(classes=512)

from face_toolbox_keras.models.parser import face_parser
fp = face_parser.FaceParser()

from face_toolbox_keras.models.detector import face_detector
fd = face_detector.FaceAlignmentDetector()

from face_toolbox_keras.models.detector.iris_detector import IrisDetector
idet = IrisDetector()
#idet.set_detector(fd)

## Upload test data

In [ ]:
import numpy as np
from utils import utils
from matplotlib import pyplot as plt

In [ ]:
from google.colab import files

In [ ]:
# Upload a source face image.
# There should be only one source face image.
fn_src = files.upload()

In [ ]:
# Upload target face images.
# Number of target face images is not restricted as long as they belong to the same identity.
fns_tar = files.upload()

In [ ]:
# Set input image path
fn_src = [k for k,v in fn_src.items()]
if len(fn_src) >= 1:
    fn_src = fn_src[0]

fns_tar = [k for k,v in fns_tar.items()]

print(fn_src)
print(fns_tar)

In [ ]:
# We can also manually assign filenmes using the following code
#fn_src = "test02.jpg"
#fns_tar = ["test01.jpg", "test03.jpg", "test04.jpg", "test05.jpg"]

## Translate faces

### Inferece

It requires additional time to load models for the first infernce.

In [ ]:
src, mask, aligned_im, (x0, y0, x1, y1), landmarks = utils.get_src_inputs(fn_src, fd, fp, idet, identity_extractor=config["identity_extractor"])
tar, emb_tar = utils.get_tar_inputs(fns_tar, fd, fv, identity_extractor=config["identity_extractor"])

In [ ]:
out = model.inference(src, mask, tar, emb_tar)

### Visualize results

Images are resized to having maximum side length of 768.

In [ ]:
plt.figure()
plt.subplot(1,2,1)
plt.imshow(src)
plt.subplot(1,2,2)
plt.imshow(tar)

In [ ]:
result_face = np.squeeze(((out[0] + 1) * 255 / 2).astype(np.uint8))
plt.imshow(result_face)

In [ ]:
result_img = utils.post_process_result(fn_src, fd, result_face, aligned_im, src, x0, y0, x1, y1, landmarks)
plt.imshow(result_img)